In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


In [ ]:
import numpy as np
import scipy.io.wavfile as wavfile
import librosa
import os
import math
from audiomentations import Compose, AddBackgroundNoise
import sys

#example usage: create_noise.py music 

if len(sys.argv) != 3:
	print("usage: create_noise.py <clean directory> <mixed_noise_directory>")
	exit()


def read_file(filename):
    wav_data, sr = librosa.load(filename, sr=16000)
    if sr != 16000:
        print(sr)
        raise ValueError('Sampling rate is expected to be 16kHz!')
    return wav_data

def write_file(signal, path):
    wavfile.write(path, 16000, signal.astype(np.float32))


def add_white_noise(signal, SNR):
    RMS_signal = np.sqrt(np.mean(signal**2))
    RMS_required = np.sqrt((RMS_signal**2)/pow(10, SNR/10))

    noise = np.random.normal(0, RMS_required, signal.shape[0])
    return signal + noise

def get_noise_from_sound(signal,noise,SNR):
    RMS_s=math.sqrt(np.mean(signal**2))
    #required RMS of noise
    RMS_n=math.sqrt(RMS_s**2/(pow(10,SNR/10)))
    
    #current RMS of noise
    RMS_n_current=math.sqrt(np.mean(noise**2))
    noise=noise*(RMS_n/RMS_n_current)
    
    return noise

def mix_noise(signal, noise_filename, SNR):
    noise_signal = read_file(noise_filename)
    noise_start = np.random.randint(0, noise_signal.shape[0] - signal.shape[0])
    noise_signal = noise_signal[noise_start : noise_start + signal.shape[0]]
    noise_signal = np.interp(noise_signal, (noise_signal.min(), noise_signal.max()), (-1.0, 1.0))

    # RMS_signal = np.sqrt(np.mean(signal**2))
    # RMS_noise = np.sqrt(np.mean(noise_signal**2))
    # RMS_required = np.sqrt((RMS_signal**2)/pow(10, SNR/10))

    # noise = noise_signal * (RMS_required / RMS_noise)
    noise = get_noise_from_sound(signal, noise_signal, SNR)
    return signal + noise



clean_dir = sys.argv[1]
if not os.path.exists(clean_dir):
    raise ValueError("Clean directory: {} does not exist".format(clean_dir))

mixed_dir = sys.argv[2]
if not os.path.exists(mixed_dir):
    raise ValueError("Mixed noise directory: {} does not exist".format(mixed_dir))

noisy_dir = clean_dir.replace(os.path.basename(os.path.normpath(clean_dir)), "noisy_" + os.path.basename(os.path.normpath(clean_dir)))
if not os.path.exists(noisy_dir):
    os.mkdir(noisy_dir)


wav_files = [
    os.path.join(clean_dir, wav) for wav in os.listdir(clean_dir)
    if wav.endswith('.wav')
]

mixed_noise_files = [
    os.path.join(mixed_dir, wav) for wav in os.listdir(mixed_dir)
    if wav.endswith('.wav')
]

print("Creating noisy signals...")
nfiles = len(wav_files)
for m, wav_file in enumerate(wav_files):
    signal = read_file(wav_file)

    file_count = 0
    head_tail = os.path.split(wav_file)
    file_ext = os.path.splitext(head_tail[1])
    file_name = "{}{}_{}{}".format(noisy_dir, file_ext[0], file_count, file_ext[1])

    white_noise = add_white_noise(signal, 15)
    white_noise = np.interp(white_noise, (white_noise.min(), white_noise.max()), (-32767, 32767))
    wavfile.write(file_name, 16000, white_noise.astype(np.int16))

    if m % 100 == 0:
        print("{}/{} files complete".format(m+1, nfiles))
        print(file_name)

    for mixed_wav in mixed_noise_files:
        #mixed_noise = mix_noise(signal, mixed_wav, 1)
        augment = Compose([ AddBackgroundNoise(sounds_path="datasets/mixed/", min_snr_in_db=25,  min_absolute_rms_in_db=-16, max_absolute_rms_in_db=-13, p=1, noise_rms='absolute')])

        mixed_noise = augment(samples=signal, sample_rate=16000)

        file_count += 1
        file_name = "{}{}_{}{}".format(noisy_dir, file_ext[0], file_count, file_ext[1])
        mixed_noise = np.interp(mixed_noise, (mixed_noise.min(), mixed_noise.max()), (-32767, 32767))
        wavfile.write(file_name, 16000, mixed_noise.astype(np.int16))







